In [1]:
import sys
import pandas as pd
import cv2 as cv
import numpy as np
from tqdm.notebook import tqdm

sys.path.insert(1, "E:\8thSem\BTP_Research\Project_2\modules")

In [2]:
import NbrRegionSegment as seg
from MarinePredator import MPA as MPA

In [3]:
#fitness function to be used in the marine predator algo
def fitness_func(wr, wg, wc, thres):
    #read the csv file
    data = pd.read_csv("../../CSV/reduced.csv")

    #fitness for each image against each predator
    fit = list()

    #fetch the image file
    for entry in tqdm(data.values):
        #read the image
        img = cv.imread("../../Dataset/" + entry[2] + ".jpg")

        gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        #LBP part
        radius = 1
        points = 8*radius 

        lbp = local_binary_pattern(gray_img, points, radius, "uniform").astype("uint8")

        #segment image
        r = seg.SegmentImage(img, lbp, thres, wr, wg, wc, True)
        r1, r2 = r

        #find the fitness level
        intra, inter = seg.GetAllSegVariance(gray_img, r1, r2.shape[0], True)

        #push the fitness into the list
        fit.append(intra + inter) #-> fitness is the sum of the inter and intra variance and is a decreasing function
    
    #return the fitness as the avg of all fitness
    return np.mean(fit)

In [5]:
#algo params for the MPA
max_iter = 30
num_agents = 15

#wr, wg, wc, thres
lb = [0, 0, 0, 10]
ub = [1, 1, 1, 50]
num_var = 4

#create a class object for the metaheuristic algorithm
mpa = MPA.MPA(num_agents, max_iter, lb, ub, num_var, fitness_func)

In [6]:
#init the mpa algo
mpa.initialize()

In [7]:
mpa.Prey.shape

(15, 4)

In [ ]:
mpa.run()